# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

# Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='~/../images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='~/../images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

# Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!


# Step 1: Load the Data/Filtering for Chosen Zipcodes

# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

# Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='~/../images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='~/../images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

# Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!


# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [1]:
ls

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

In [3]:
df = pd.read_csv('zillow_data.csv')
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14723 entries, 0 to 14722
Columns: 272 entries, RegionID to 2018-04
dtypes: float64(219), int64(49), object(4)
memory usage: 30.6+ MB


In [5]:
df.drop('RegionID', axis=1, inplace=True)

In [6]:
df.drop('SizeRank', axis=1, inplace=True)

In [7]:
df[df['City'] == 'Kansas City'].shape

(37, 270)

In [34]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionName', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})


# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
    # prepare training dataset
    train_size = int(len(X) * 0.66)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]
    # make predictions
    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit(disp=0)
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
    # calculate out of sample error
    error = mean_squared_error(test, predictions)
    return error
 
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
    dataset = dataset.astype('float32')
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                try:
                    mse = evaluate_arima_model(dataset, order)
                    if mse < best_score:
                        best_score, best_cfg = mse, order
                    print('ARIMA%s MSE=%.3f' % (order,mse))
                except:
                    continue
    print('Best ARIMA%s MSE=%.3f' % (best_cfg, best_score))
    return best_cfg




In [11]:
for index, x in df[df['City'] == 'Kansas City'].iterrows():
    print(x['RegionName'])

64119
64114
64151
64111
66102
64131
66104
64155
66109
64134
64116
66106
64157
64110
64117
66103
64112
66112
64154
64108
64113
64106
64124
64137
64105
64129
64123
66111
64126
64156
64153
64158
64145
64136
64125
64139
64146


In [14]:
series = melt_data(df.loc[[2919]])
model= ARIMA(series, order=(1,1,1))
model_fit= model.fit()

C:\Users\luigi\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)
C:\Users\luigi\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:191: FutureWarning: Creating a DatetimeIndex by passing range endpoints is deprecated.  Use `pandas.date_range` instead.
  start=index[0], end=index[-1], freq=freq)
C:\Users\luigi\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)


In [16]:
model_fit.forecast(steps=12)[0]

array([143073.61560612, 143331.26046148, 143577.72436054, 143816.36058853,
       144049.51674551, 144278.83636184])

In [29]:
six_months = model_fit.forecast(steps=36)[0][-1]
(six_months - today)/today

0.05679732209422842

In [ ]:
series

In [26]:
today = series.iloc[-1].value

In [38]:
roi_list = {}
for index, x in df[df['City'] == 'Kansas City'].iterrows():
    print(x['RegionName'])
    series = melt_data(df.loc[[index]])
    # evaluate parameters
    p_values = [0, 1, 2]
    d_values = range(0, 2)
    q_values = range(0, 2)
    warnings.filterwarnings("ignore")
    
    order = evaluate_models(series.values, p_values, d_values, q_values)
    
    model= ARIMA(series, order=order)
    model_fit= model.fit()
    six_months = model_fit.forecast(steps=36)[0][-1]
    today = series.iloc[-1].value
    roi = (six_months - today)/today
    roi_list[x['RegionName']] = roi

64119
ARIMA(0, 0, 0) MSE=93611590.076
ARIMA(0, 0, 1) MSE=24344135.774
ARIMA(0, 1, 0) MSE=875810.127
ARIMA(0, 1, 1) MSE=351653.230
ARIMA(1, 0, 0) MSE=921897.011
ARIMA(1, 1, 0) MSE=337473.443
ARIMA(1, 1, 1) MSE=256744.799
ARIMA(2, 0, 0) MSE=338327.407
ARIMA(2, 0, 1) MSE=259557.192
ARIMA(2, 1, 0) MSE=291309.276
ARIMA(2, 1, 1) MSE=259864.982
Best ARIMA(1, 1, 1) MSE=256744.799
64114
ARIMA(0, 0, 0) MSE=559210333.009
ARIMA(0, 0, 1) MSE=144359078.033
ARIMA(0, 1, 0) MSE=881044.928
ARIMA(0, 1, 1) MSE=322385.536
ARIMA(1, 0, 0) MSE=1130423.428
ARIMA(1, 1, 0) MSE=182842.663
ARIMA(1, 1, 1) MSE=157165.738
ARIMA(2, 0, 0) MSE=185511.413
ARIMA(2, 1, 0) MSE=170707.770
ARIMA(2, 1, 1) MSE=159283.549
Best ARIMA(1, 1, 1) MSE=157165.738
64151
ARIMA(0, 0, 0) MSE=348965340.874
ARIMA(0, 0, 1) MSE=90068757.065
ARIMA(0, 1, 0) MSE=1476214.284
ARIMA(0, 1, 1) MSE=555361.512
ARIMA(1, 0, 0) MSE=1611215.523
ARIMA(1, 1, 0) MSE=589199.533
ARIMA(1, 1, 1) MSE=372531.470
ARIMA(2, 0, 0) MSE=593654.932
ARIMA(2, 0, 1) MSE=37947

ARIMA(1, 1, 1) MSE=225963.580
ARIMA(2, 0, 0) MSE=315311.063
ARIMA(2, 0, 1) MSE=227134.652
ARIMA(2, 1, 0) MSE=281724.451
ARIMA(2, 1, 1) MSE=231416.459
Best ARIMA(1, 1, 1) MSE=225963.580
64105
ARIMA(0, 0, 0) MSE=233439718.259
ARIMA(0, 0, 1) MSE=59342038.307
ARIMA(0, 1, 0) MSE=1685229.670
ARIMA(0, 1, 1) MSE=734414.826
ARIMA(1, 0, 0) MSE=1585561.399
ARIMA(1, 1, 0) MSE=1016577.926
ARIMA(1, 1, 1) MSE=682609.006
ARIMA(2, 0, 0) MSE=1018268.790
ARIMA(2, 0, 1) MSE=686752.896
ARIMA(2, 1, 0) MSE=856858.046
ARIMA(2, 1, 1) MSE=676616.762
Best ARIMA(2, 1, 1) MSE=676616.762
64129
ARIMA(0, 0, 0) MSE=116257594.074
ARIMA(0, 0, 1) MSE=28874146.911
ARIMA(0, 1, 0) MSE=727670.750
ARIMA(0, 1, 1) MSE=294036.101
ARIMA(1, 0, 0) MSE=705104.698
ARIMA(1, 1, 0) MSE=357629.389
ARIMA(1, 1, 1) MSE=251863.181
ARIMA(2, 0, 1) MSE=250819.286
ARIMA(2, 1, 0) MSE=323110.471
ARIMA(2, 1, 1) MSE=256519.700
Best ARIMA(2, 0, 1) MSE=250819.286
64123
ARIMA(0, 0, 0) MSE=23137869.162
ARIMA(0, 0, 1) MSE=5809465.250
ARIMA(0, 1, 0) MSE=6

In [42]:
roi_list

{64119: 0.05679732209422842,
 64114: 0.1192970729762698,
 64151: 0.05905026321941397,
 64111: 0.09693136094924668,
 66102: 0.01407573960837967,
 64131: 0.04546284263701976,
 66104: 0.0017268933293670607,
 64155: 0.06257263719843865,
 66109: 0.07817196811403077,
 64134: 0.04056394230247112,
 64116: 0.07975137309361202,
 66106: 0.1373985358861824,
 64157: 0.06528310078178046,
 64110: 0.08353030324922359,
 64117: 0.06822308914977174,
 66103: 0.10155508045775981,
 64112: 0.10592691843828653,
 66112: 0.08488559860951322,
 64154: 0.04150743227539524,
 64108: 0.03203111596063606,
 64113: 0.08080219700287117,
 64106: 0.14737096179258374,
 64124: 0.0067724159021344005,
 64137: 0.06481306716880608,
 64105: 0.06870997231295099,
 64129: -0.01684479110929328,
 64123: 0.05745865294676608,
 66111: 0.06020684041017004,
 64126: 0.06094194217722193,
 64156: 0.06920193050296697,
 64153: 0.05623209319876416,
 64158: 0.06907822253949673,
 64145: 0.09720449413548049,
 64136: 0.04962982541605204,
 64125: 0.0

In [47]:
s = [(k, roi_list[k]) for k in sorted(roi_list, key=roi_list.get, reverse=True)]
for k, v in s:
    print(k, v)

64106 0.14737096179258374
66106 0.1373985358861824
64114 0.1192970729762698
64112 0.10592691843828653
66103 0.10155508045775981
64145 0.09720449413548049
64111 0.09693136094924668
64139 0.09670521678169679
66112 0.08488559860951322
64110 0.08353030324922359
64113 0.08080219700287117
64116 0.07975137309361202
66109 0.07817196811403077
64156 0.06920193050296697
64158 0.06907822253949673
64105 0.06870997231295099
64117 0.06822308914977174
64157 0.06528310078178046
64137 0.06481306716880608
64155 0.06257263719843865
64126 0.06094194217722193
66111 0.06020684041017004
64151 0.05905026321941397
64123 0.05745865294676608
64119 0.05679732209422842
64125 0.05666884598152787
64153 0.05623209319876416
64136 0.04962982541605204
64131 0.04546284263701976
64146 0.044572768695209576
64154 0.04150743227539524
64134 0.04056394230247112
64108 0.03203111596063606
66102 0.01407573960837967
64124 0.0067724159021344005
66104 0.0017268933293670607
64129 -0.01684479110929328


In [50]:
s[:5]

[(64106, 0.14737096179258374),
 (66106, 0.1373985358861824),
 (64114, 0.1192970729762698),
 (64112, 0.10592691843828653),
 (66103, 0.10155508045775981)]

In [ ]:
pd.DataFrame(df.loc[index])

In [ ]:
melt_data(df.loc[[index]])

In [ ]:
melt_data(pd.DataFrame(x))

# Step 2: Data Preprocessing

In [ ]:
def get_datetimes(df):
    return pd.to_datetime(df.columns.values[1:], format='%Y-%m')

# Step 3: EDA and Visualization

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

# Step 4: Reshape from Wide to Long Format

In [ ]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionName', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

In [ ]:
melt_data(df)

In [ ]:
chicago = df[df['RegionName'] == 60657]
chicago

In [ ]:
chicago_time.describe()

In [ ]:
chicago_time = melt_data(chicago)

In [ ]:
chicago_time.plot(figsize=(12,8));

In [ ]:
def test_stationarity(timeseries):
    #Determing rolling statistics
    rolmean = timeseries.rolling(window=12).mean()
    rolstd = timeseries.rolling(window=12).std()

    #Plot rolling statistics:
    plt.figure(figsize=(12,8))
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)

In [ ]:
test_stationarity(chicago_time)

In [ ]:
chicago_diff = chicago_time.diff()
chicago_diff.plot(figsize=(12,8));

In [ ]:
chicago_diff1 = chicago_diff.diff()
chicago_diff1.plot(figsize=(12,8));

In [ ]:
chicago_diff2 = chicago_diff1.diff()
chicago_diff2.plot(figsize=(12,8));

In [ ]:
chicago_2009_onwards = chicago_time['2009': '2018']
print(chicago_2009_onwards.head())

In [ ]:
chicago_2009_onwards.plot(figsize=(12,8))

In [ ]:
chicago_2009_onwards.plot(figsize = (20,6), style = ".b");

In [ ]:
year_groups = chicago_2009_onwards.groupby(pd.Grouper(freq ='A'))

In [ ]:
len(year_groups)

In [ ]:
for yr, group in year_groups:
    print(group.values.ravel())
    

In [ ]:
chicago_annual

In [ ]:
chicago_2009_onwards.head()

In [ ]:
chicago_2009_onwards['Month'] = chicago_2009_onwards.index.month

In [ ]:
chicago_2009_onwards.head()

In [ ]:
chicago_2009_onwards.groupby('Month')['value'].mean().plot(kind='line')

In [ ]:
chicago_annual = pd.DataFrame()

for yr, group in year_groups:
    print(yr)
    chicago_annual[yr] = group.values.ravel()
    
# Plot the yearly groups as subplots
chicago_annual.plot(figsize = (13,8), subplots=True, legend=True);

# Step 5: ARIMA Modeling

In [ ]:
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
	# prepare training dataset
	train_size = int(len(X) * 0.66)
	train, test = X[0:train_size], X[train_size:]
	history = [x for x in train]
	# make predictions
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit(disp=0)
		yhat = model_fit.forecast()[0]
		predictions.append(yhat)
		history.append(test[t])
	# calculate out of sample error
	error = mean_squared_error(test, predictions)
	return error
 
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					mse = evaluate_arima_model(dataset, order)
					if mse < best_score:
						best_score, best_cfg = mse, order
					print('ARIMA%s MSE=%.3f' % (order,mse))
				except:
					continue
	print('Best ARIMA%s MSE=%.3f' % (best_cfg, best_score))
 
series = chicago_time
# evaluate parameters
p_values = [0, 1, 2, 4, 6, 8, 10]
d_values = range(0, 3)
q_values = range(0, 3)
warnings.filterwarnings("ignore")
evaluate_models(series.values, p_values, d_values, q_values)

# Step 2: Data Preprocessing

In [ ]:
def get_datetimes(df):
    return pd.to_datetime(df.columns.values[1:], format='%Y-%m')

# Step 3: EDA and Visualization

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

# Step 4: Reshape from Wide to Long Format

In [ ]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionName', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

# Step 5: ARIMA Modeling

# Step 6: Interpreting Results